In [1]:
from astropy.io import fits
import h5py
import numpy as np

In [2]:
folder = '/global/projecta/projectdirs/lsst/groups/WL/users/jprat/gaussian_sims_srdnzs_fullsky/111221/'

## Compute what declination cut we need to mimic LSST Y1 area

In [ ]:
def deg2_to_rad2(deg2):
    return deg2*(np.pi/180)**2

def h_func(A, r):
    return (2*np.pi*r**2-A)/(2*np.pi*r) # area of a slice is A= 2*pi*r(r-h)

def dec_lim_func(h, r):
    return np.arccos((r-h)/r)

# We want to get an area of 12,300 deg^2 (LSST Y1)
A = deg2_to_rad2(12300)
h = h_func(A, r=1)
dec_lim = -dec_lim_func(h, r=1)*180/np.pi # negative because lsst is in the south
print(h)
print(dec_lim)

## Now mask the source catalogs

In [ ]:
for zbin in range(1,6):
    filename = 'shearcat_shearalm_zbin%d_ra_dec_g1_g2_withshapenoise'%zbin
    cat = np.load(folder + filename + '.npy')
    ra, dec, g1, g2 = cat.T
    mask = (dec>dec_lim)&(dec<0)
    print(len(ra))
    ra_cut, dec_cut, g1_cut, g2_cut = ra[mask], dec[mask], g1[mask], g2[mask]
    print(len(ra_cut))
    np.save(folder + '/area12300/' + filename + '_areacut', (ra_cut,dec_cut,g1_cut,g2_cut))

In [ ]:

for zbin in range(1,6):
    filename = 'shearcat_shearalm_zbin%d_ra_dec_g1_g2'%zbin
    cat = np.load(folder + filename + '.npy')
    ra, dec, g1, g2 = cat.T
    mask = (dec>dec_lim)&(dec<0)
    print(len(ra))
    ra_cut, dec_cut, g1_cut, g2_cut = ra[mask], dec[mask], g1[mask], g2[mask]
    print(len(ra_cut))
    np.save(folder + '/area12300/' + filename + '_areacut', (ra_cut,dec_cut,g1_cut,g2_cut))

In [ ]:
filename = 'galcat_%d.npy'%zbin
lenscat = np.load(folder + 'galcat_2.npy')

## Now cut the lens galaxy catalog

In [ ]:
for zbin in range(1,6):
    filename = 'galcat_%d'%zbin
    lenscat = np.load(folder + filename + '.npy')
    ra, dec = lenscat.T
    mask = (dec>dec_lim)&(dec<0)
    print(len(ra))
    ra_cut, dec_cut = ra[mask], dec[mask]
    print(len(ra_cut))
    np.save(folder + '/area12300/' + filename + '_areacut', (ra_cut,dec_cut))

# Now joint source files into a single one, to inject to TXPipe, adding columns it needs

In [3]:
# Columns TXPipe needs from the cosmo simulation
cols = ['mag_true_u_lsst', 'mag_true_g_lsst', 
                'mag_true_r_lsst', 'mag_true_i_lsst', 
                'mag_true_z_lsst', 'mag_true_y_lsst',
                'ra', 'dec',
                'ellipticity_1_true', 'ellipticity_2_true',
                'shear_1', 'shear_2',
                'size_true',
                'galaxy_id',
                'redshift_true',
                ]


In [17]:
filepath_save = folder + '/area12300/' + 'shearcat_shearalm_allbins_ra_dec_g1_g2_z_withshapenoise' + '_areacut'
filepath_save_fake = folder + '/area12300/' + 'shearcat_shearalm_allbins_fake_input' + '_areacut'

In [5]:
#assign a fake redshift within z lims so that TXPipe assigns it properly when source selector splits in bins
source_zbin_edges = np.array([0.19285902, 0.40831394, 0.65503818, 0.94499109, 1.2947086, 1.72779632, 2.27855242, 3. ]) # 7 bins
z_fake =[0.3, 0.5, 0.8, 1., 1.4, 1.9, 2.5]
print(z_fake)

[0.3, 0.5, 0.8, 1.0, 1.4, 1.9, 2.5]


In [10]:
ras, decs, g1s, g2s, zs = [], [], [], [], []

for zbin in range(1,6):
    filename = 'shearcat_shearalm_zbin%d_ra_dec_g1_g2_withshapenoise'%zbin
    cat = np.load(folder + '/area12300/' + filename + '_areacut'+'.npy')
    ra, dec, g1, g2 = cat
    ras.extend(ra)
    decs.extend(dec)
    g1s.extend(g1)   
    g2s.extend(g2)
    zs.extend([z_fake[zbin-1]]*len(ra))
    
ras, decs, g1s, g2s, zs = np.array(ras), np.array(decs), np.array(g1s), np.array(g2s), np.array(zs)   
np.save(filepath_save, (ras,decs,g1s,g2s,zs))

In [13]:
cat_all = np.load(filepath_save + '.npy')
nobj = len(cat_all[0])

In [14]:
# Now fake the rest of the columns we need matching the lenth

In [15]:
len(ras), len(decs), len(g1s), len(g2s), len(zs)

(530494785, 530494785, 530494785, 530494785, 530494785)

In [11]:
# remaining columns to save:
cols = ['mag_true_u_lsst', 'mag_true_g_lsst', 
 'mag_true_r_lsst', 'mag_true_i_lsst', 
'mag_true_z_lsst', 'mag_true_y_lsst',
'ellipticity_1_true', 'ellipticity_2_true',
'size_true','galaxy_id']


In [16]:
m_u = np.random.uniform(low=20, high=30, size=(nobj,))
m_g = np.random.uniform(low=20, high=30, size=(nobj,))
m_r = np.random.uniform(low=20, high=30, size=(nobj,))
m_i = np.random.uniform(low=20, high=30, size=(nobj,))
m_z = np.random.uniform(low=20, high=30, size=(nobj,))
m_y = np.random.uniform(low=20, high=30, size=(nobj,))
etrue1 = np.random.uniform(low=0, high=0.5, size=(nobj,))
etrue2 = np.random.uniform(low=0, high=0.5, size=(nobj,))
size = np.random.uniform(low=2., high=10., size=(nobj,))
galaxy_id = np.arange(0,nobj, 1, dtype=int)

In [18]:
np.save(filepath_save_fake, (m_u,m_g, m_r, m_i, m_z, m_y, etrue1, etrue2, size, galaxy_id))